In [210]:
#import libraries
import pandas as pd
import numpy as np
import re
from rapidfuzz.fuzz import ratio
from collections import Counter
import ast 
from itertools import chain

from anime_list import all_watched_anime

In [211]:
#load watched anime list
anime_list = all_watched_anime

In [212]:
#load df
raw_df = pd.read_csv("anime_fran_data.csv")
raw_df.head()

,mal_id,url,approved,titles,title,title_english,title_japanese,title_synonyms,type,source,...,aired.prop.to.day,aired.prop.to.month,aired.prop.to.year,aired.string,broadcast.day,broadcast.time,broadcast.timezone,broadcast.string,relation_type,franchise_id
0,4970,https://myanimelist.net/anime/4970/Afro_Samura...,True,"[{'type': 'Default', 'title': 'Afro Samurai: R...",Afro Samurai: Resurrection,Afro Samurai: Resurrection,アフロサムライ Resurrection,[],Movie,Original,...,NaN,NaN,NaN,"Jan 25, 2009",NaN,NaN,NaN,NaN,Sequel,1292
1,13709,https://myanimelist.net/anime/13709/Afro_Samur...,True,"[{'type': 'Default', 'title': 'Afro Samurai Mo...",Afro Samurai Movie,Afro Samurai the Movie,AFRO SAMURAI 劇場版,[],Movie,Original,...,NaN,NaN,NaN,"Oct 27, 2007",NaN,NaN,NaN,NaN,Prequel,1292
2,27939,https://myanimelist.net/anime/27939/Afro_Samur...,True,"[{'type': 'Default', 'title': 'Afro Samurai Pi...",Afro Samurai Pilot,NaN,アフロサムライ,[],Special,Original,...,NaN,NaN,NaN,"Jan 1, 2003",NaN,NaN,NaN,NaN,Other,1292
3,1292,https://myanimelist.net/anime/1292/Afro_Samurai,True,"[{'type': 'Default', 'title': 'Afro Samurai'},...",Afro Samurai,Afro Samurai,アフロサムライ,[],TV,Manga,...,4.0,5.0,2007.0,"May 3, 2007 to May 4, 2007",NaN,NaN,NaN,Unknown,main,1292
4,25777,https://myanimelist.net/anime/25777/Shingeki_n...,True,"[{'type': 'Default', 'title': 'Shingeki no Kyo...",Shingeki no Kyojin Season 2,Attack on Titan Season 2,進撃の巨人 Season2,[],TV,Manga,...,17.0,6.0,2017.0,"Apr 1, 2017 to Jun 17, 2017",Saturdays,22:00,Asia/Tokyo,Saturdays at 22:00 (JST),Sequel,16498


## Initial Inspection

In [213]:
#inspect df
raw_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 596 entries, 0 to 595
Data columns (total 61 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   mal_id                            596 non-null    int64  
 1   url                               596 non-null    object 
 2   approved                          596 non-null    bool   
 3   titles                            596 non-null    object 
 4   title                             596 non-null    object 
 5   title_english                     475 non-null    object 
 6   title_japanese                    594 non-null    object 
 7   title_synonyms                    596 non-null    object 
 8   type                              591 non-null    object 
 9   source                            596 non-null    object 
 10  episodes                          596 non-null    float64
 11  status                            596 non-null    object 
 12  airing  

In [214]:
raw_df.dtypes

mal_id                 int64
url                   object
approved                bool
titles                object
title                 object
                       ...  
broadcast.time        object
broadcast.timezone    object
broadcast.string      object
relation_type         object
franchise_id           int64
Length: 61, dtype: object

In [215]:
raw_df.describe()

,mal_id,episodes,score,scored_by,rank,popularity,members,favorites,year,aired.prop.from.day,aired.prop.from.month,aired.prop.from.year,aired.prop.to.day,aired.prop.to.month,aired.prop.to.year,franchise_id
count,596.000000,596.000000,567.000000,5.670000e+02,516.000000,596.000000,5.960000e+02,596.000000,178.000000,581.000000,581.000000,581.000000,245.000000,245.000000,245.000000,596.000000
mean,32067.110738,12.979866,7.376155,2.005325e+05,3243.443798,3861.536913,3.323593e+05,6736.692953,2015.252809,13.163511,6.187608,2015.134251,20.346939,6.665306,2015.546939,14020.854027
std,20192.716618,56.446178,0.853092,4.148595e+05,3210.828104,3838.457109,6.281958e+05,24264.224536,8.107897,9.150228,3.529812,7.676602,8.731775,3.392709,7.246455,18169.870527
min,1.000000,0.000000,4.720000,1.720000e+02,1.000000,1.000000,3.960000e+02,0.000000,1969.000000,1.000000,1.000000,1968.000000,1.000000,1.000000,1969.000000,1.000000
25%,10655.000000,1.000000,6.720000,7.728500e+03,492.000000,861.000000,1.875675e+04,27.000000,2010.250000,5.000000,3.000000,2010.000000,16.000000,3.000000,2011.000000,232.000000
50%,36239.500000,1.000000,7.330000,3.539800e+04,2330.000000,2773.500000,7.409450e+04,153.500000,2017.000000,11.000000,7.000000,2017.000000,24.000000,6.000000,2017.000000,1575.000000
75%,50458.000000,12.000000,8.095000,1.700630e+05,5500.000000,5428.250000,3.133620e+05,1635.250000,2021.000000,21.000000,9.000000,2021.000000,27.000000,9.000000,2021.000000,32615.000000
max,62352.000000,1140.000000,9.290000,2.971139e+06,20004.000000,18975.000000,4.217418e+06,242292.000000,2026.000000,31.000000,12.000000,2026.000000,31.000000,12.000000,2025.000000,57334.000000


## Deduplication

In [216]:
#sort by franchise_id ascending
df_sorted = raw_df.sort_values("franchise_id", ascending=True)

#drop duplicates on all other columns
keys = [c for c in raw_df.columns if c not in ("relation_type","franchise_id")]
df_deduped = df_sorted.drop_duplicates(subset=keys, keep="first")
print(f"{len(df_sorted)},{len(df_deduped)}")

596,546


In [217]:
#Remove unrelated anime

#parse string into actual Python objects
def parse_titles(x):
    if isinstance(x, str):
        try:
            return ast.literal_eval(x)
        except Exception as e:
            # if parsing fails, return empty list so it won’t match
            print("⚠️ Could not parse titles:", x)
            return []
    return x  # already a list

df_deduped["titles"] = df_deduped["titles"].apply(parse_titles)

# Normalization helper
def normalize(text):
    return re.sub(r'\W+', '', str(text).lower())

normalized_watched = [normalize(name) for name in all_watched_anime]

# 3)fuzzy‐match across every title
def is_related_any_title_fuzzy(titles_list):
    if not isinstance(titles_list, list):
        return False

    for entry in titles_list:
        raw = entry.get("title", "")
        if not raw:
            continue

        norm = normalize(raw)
        for watched in normalized_watched:
            sim = ratio(norm, watched)
            # match by substring OR high fuzzy similarity
            if watched in norm or norm in watched or sim > 85:
                # debug print:
                # print(f"✅ {raw} ↔ {watched} ({sim:.1f})")
                return True

    return False

# Apply mask and split
mask = df_deduped["titles"].apply(is_related_any_title_fuzzy)
filtered_df = df_deduped[mask].reset_index(drop=True)
dropped_df  = df_deduped[~mask].reset_index(drop=True)

print(f"✅ KEPT: {len(filtered_df)} rows")
print(f"❌ DROPPED: {len(dropped_df)} rows")
dropped_df.to_csv("anime_dropped.csv", index=False)


✅ KEPT: 509 rows
❌ DROPPED: 37 rows


/var/folders/r3/20vxfkdn06q1gs5plztt2_bm0000gn/T/ipykernel_8518/1402065278.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_deduped["titles"] = df_deduped["titles"].apply(parse_titles)


In [218]:
#remove crossovers
collabs = ["collab", "crossover", "x "]

def is_collab_title(titles_list):
    if not isinstance(titles_list, list):
        return False
    for entry in titles_list:
        raw = entry.get("title", "")
        norm = normalize(raw)
        for word in collabs:
            if word in norm:
                return True
    return False

after_fuzzy = filtered_df  

# collab rows mask
mask_collab = after_fuzzy["titles"].apply(is_collab_title)

# Remove collabs
collab_filtered = after_fuzzy[~mask_collab].reset_index(drop=True)
collab_titles = after_fuzzy[mask_collab].reset_index(drop=True)

print(f"✅ FINAL KEPT: {len(collab_filtered)} rows")
print(f"❌ BLACKLISTED REMOVED: {len(collab_titles)} rows")
print(collab_titles["title"]) #sanity check


✅ FINAL KEPT: 507 rows
❌ BLACKLISTED REMOVED: 2 rows
0                                     Toriko
1    Gintama x Meta Quest 3 Collaboration CM
Name: title, dtype: object


In [219]:
#map franchsise name
collab_filtered['franchise_name'] = collab_filtered['franchise_id'].map(collab_filtered.set_index('mal_id')['title_english'])
collab_filtered['franchise_name'] = collab_filtered['franchise_name'].fillna(collab_filtered['franchise_id'].map(collab_filtered.set_index('mal_id')['title']))

In [220]:
#check for null values
collab_filtered.isnull().sum()

mal_id                  0
url                     0
approved                0
titles                  0
title                   0
                     ... 
broadcast.timezone    358
broadcast.string      346
relation_type           0
franchise_id            0
franchise_name          0
Length: 62, dtype: int64

In [221]:
#check null scores - here it's mainly unaired animes that aren't scored (except 1) this can be taken care of later
collab_filtered[collab_filtered["score"].isnull()]

,mal_id,url,approved,titles,title,title_english,title_japanese,title_synonyms,type,source,...,aired.prop.to.month,aired.prop.to.year,aired.string,broadcast.day,broadcast.time,broadcast.timezone,broadcast.string,relation_type,franchise_id,franchise_name
21,54688,https://myanimelist.net/anime/54688/Naruto_Shi...,True,"[{'type': 'Default', 'title': 'Naruto (Shinsak...",Naruto (Shinsaku Anime),NaN,ナルト新作アニメ,[],TV,Manga,...,NaN,NaN,Not available,NaN,NaN,NaN,Unknown,Other,20,Naruto
25,54687,https://myanimelist.net/anime/54687/Boruto__Na...,True,"[{'type': 'Default', 'title': 'Boruto: Naruto ...",Boruto: Naruto Next Generations Part 2,NaN,BORUTO -NARUTO NEXT GENERATIONS- (第二部),[],TV,Manga,...,NaN,NaN,Not available,NaN,NaN,NaN,Unknown,Sequel,20,Naruto
107,57557,https://myanimelist.net/anime/57557/The_One_Piece,True,"[{'type': 'Default', 'title': 'The One Piece'}...",The One Piece,NaN,THE ONE PIECE,[],ONA,Manga,...,NaN,NaN,Not available,NaN,NaN,NaN,NaN,Alternative Version,21,One Piece
135,54916,https://myanimelist.net/anime/54916/Cardcaptor...,True,"[{'type': 'Default', 'title': 'Cardcaptor Saku...",Cardcaptor Sakura: Clear Card-hen (Zoku-hen),NaN,カードキャプターさくら クリアカード編 (続編),['Cardcaptor Sakura: Clear Card (Sequel)'],NaN,Manga,...,NaN,NaN,Not available,NaN,NaN,NaN,NaN,Sequel,232,Cardcaptor Sakura
169,61269,https://myanimelist.net/anime/61269/Digimon_Be...,True,"[{'type': 'Default', 'title': 'Digimon Beatbre...",Digimon Beatbreak,NaN,DIGIMON BEATBREAK（デジモンビートブレイク）,[],TV,Other,...,NaN,NaN,"Oct 5, 2025 to ?",Sundays,09:00,Asia/Tokyo,Sundays at 09:00 (JST),Other,552,Digimon: Digital Monsters
203,61202,https://myanimelist.net/anime/61202/Mononoke_M...,True,"[{'type': 'Default', 'title': 'Mononoke Movie:...",Mononoke Movie: Dai San Shou - Hebigami,NaN,劇場版モノノ怪 第三章 蛇神,['Mononoke the Movie: Chapter III - Snake God'],Movie,Original,...,NaN,NaN,2026,NaN,NaN,NaN,NaN,Sequel,586,Ayakashi: Samurai Horror Tales
210,61469,https://myanimelist.net/anime/61469/JoJo_no_Ki...,True,"[{'type': 'Default', 'title': 'JoJo no Kimyou ...",JoJo no Kimyou na Bouken Part 7: Steel Ball Run,NaN,ジョジョの奇妙な冒険 Part7 STEEL BALL RUN,"[""JoJo's Bizarre Adventure Part 7: Steel Ball ...",NaN,Manga,...,NaN,NaN,Not available,NaN,NaN,NaN,NaN,Sequel,665,JoJo's Bizarre Adventure (OVA)
251,62277,https://myanimelist.net/anime/62277/Gintama_Mo...,True,"[{'type': 'Default', 'title': 'Gintama Movie 3...",Gintama Movie 3: Yoshiwara Daienjou,NaN,新劇場版 銀魂 -吉原大炎上-,['Gintama Movie 3: Yoshiwara in Flames'],Movie,Manga,...,NaN,NaN,2026,NaN,NaN,NaN,NaN,Alternative Version,918,Gintama
323,53529,https://myanimelist.net/anime/53529/Sword_Art_...,True,"[{'type': 'Default', 'title': 'Sword Art Onlin...",Sword Art Online (Original Movie),NaN,ソードアート・オンライン (オリジナル劇場版),[],Movie,Light novel,...,NaN,NaN,Not available,NaN,NaN,NaN,NaN,Other,11757,Sword Art Online
394,49233,https://myanimelist.net/anime/49233/Youjo_Senk...,True,"[{'type': 'Default', 'title': 'Youjo Senki II'...",Youjo Senki II,Saga of Tanya the Evil II,幼女戦記 II,['The Military Chronicles of a Little Girl II'...,TV,Light novel,...,NaN,NaN,Not available,NaN,NaN,NaN,Unknown,Sequel,32615,Saga of Tanya the Evil


## Transformations

In [222]:
df_trans = collab_filtered.copy()

In [223]:
#start year and end year
df_trans.loc[:, 'start_year'] = df_trans['aired.from'].fillna('').str[:4].replace('', '')
df_trans.loc[:, 'end_year']   = df_trans['aired.to'].fillna('').str[:4].replace('', '')

In [224]:
#filling in missing end years
df_trans.loc[df_trans["episodes"] == 1, "end_year"] = df_trans["start_year"]

In [225]:
#clean up ratings (helpful during viz)
rating_group = {
    "G - All Ages": "G",
    "PG - Children": "PG",
    "PG-13 - Teens 13 or older": "PG-13",
    "R+ - Mild Nudity": "R - 17+",
    "R - 17+ (violence & profanity)": "R - 17+"
}

df_trans["rating_cleaned"] = df_trans["rating"].map(rating_group)

In [226]:
#add a numeric rating column
rating_no = {
    "G": 0,
    "PG": 1,
    "PG-13": 2,
    "R - 17+" : 3,}

df_trans["rating_num"] = df_trans["rating_cleaned"].map(rating_no)

In [227]:
#convert duration to mins 
def convert_to_minutes(text):
    if pd.isna(text):
        return np.nan

    hours = 0
    minutes = 0

    if 'hr' in text:
        try:
            hr_part = text.split('hr')[0].strip().split()[-1]
            hours = int(hr_part)
        except:
            pass

    if 'min' in text:
        try:
            min_part = text.split('min')[0].strip().split()[-1]
            minutes = int(min_part)
        except:
            pass

    return hours * 60 + minutes

df_trans['duration_min'] = df_trans['duration'].apply(convert_to_minutes)

In [228]:
#extract unique values from strings
cols = ["genres","demographics","studios","themes","producers","licensors"]

def extract_names(cell):
    # if string, parse it into a Python object
    if isinstance(cell, str):
        try:
            cell = ast.literal_eval(cell)
        except Exception:
            return ""
    # if list, join all the 'name' fields
    if isinstance(cell, list):
        return ", ".join(item.get("name", "") for item in cell)
    # 3) otherwise, give empty
    return ""

# parse+extract in one go
for col in cols:
    new_col = f"{col}_clean"
    df_trans[new_col] = df_trans[col].apply(extract_names)

#sanity check
print(df_trans[[f"{col}_clean" for col in cols]].head())

                          genres_clean demographics_clean   studios_clean  \
0        Action, Award Winning, Sci-Fi                            Sunrise   
1                       Action, Sci-Fi                              Bones   
2                               Comedy                            Sunrise   
3                               Sci-Fi                            Sunrise   
4  Action, Adventure, Fantasy, Romance            Shounen  Studio Pierrot   

        themes_clean                                    producers_clean  \
0  Adult Cast, Space  Bandai Visual, Victor Entertainment, Audio Pla...   
1  Adult Cast, Space                             Sunrise, Bandai Visual   
2                                                                         
3  Adult Cast, Space                                      Bandai Visual   
4                                                    TV Tokyo, Shueisha   

                           licensors_clean  
0                               Funimatio

## Filter out unnecessary values

In [229]:
df_clean = df_trans.copy()

In [230]:
#drop unaired titles
df_clean = df_clean[~(df_clean["status"] == 'Not yet aired')] #can be used to look at currently airing/ongoing shows

In [231]:
#drop unrelated types
df_clean = df_clean[~df_clean["type"].isin(["Music","CM","PV"])]

In [232]:
df_clean[df_clean["type"].isin(["OVA"])]["duration_min"].describe()

count     26.000000
mean      32.576923
std       26.894123
min        3.000000
25%       24.000000
50%       26.500000
75%       29.750000
max      117.000000
Name: duration_min, dtype: float64

In [233]:
df_clean[df_clean["type"].isin(["ONA"])]["duration_min"].describe()

count    39.000000
mean     16.358974
std      13.129892
min       1.000000
25%       5.500000
50%      16.000000
75%      24.000000
max      66.000000
Name: duration_min, dtype: float64

In [234]:
#Remove anything with duration <5
df_clean = df_clean[~(df_clean["duration_min"] < 5)]

In [235]:
df_clean = df_clean.dropna(subset=["score"])

In [236]:
len(df_clean)

424

In [237]:
df_clean

,mal_id,url,approved,titles,title,title_english,title_japanese,title_synonyms,type,source,...,end_year,rating_cleaned,rating_num,duration_min,genres_clean,demographics_clean,studios_clean,themes_clean,producers_clean,licensors_clean
0,1,https://myanimelist.net/anime/1/Cowboy_Bebop,True,"[{'type': 'Default', 'title': 'Cowboy Bebop'},...",Cowboy Bebop,Cowboy Bebop,カウボーイビバップ,[],TV,Original,...,1999,R - 17+,3.0,24,"Action, Award Winning, Sci-Fi",,Sunrise,"Adult Cast, Space","Bandai Visual, Victor Entertainment, Audio Pla...",Funimation
1,5,https://myanimelist.net/anime/5/Cowboy_Bebop__...,True,"[{'type': 'Default', 'title': 'Cowboy Bebop: T...",Cowboy Bebop: Tengoku no Tobira,Cowboy Bebop: The Movie,カウボーイビバップ 天国の扉,"[""Cowboy Bebop: Knockin' on Heaven's Door""]",Movie,Original,...,2001,R - 17+,3.0,115,"Action, Sci-Fi",,Bones,"Adult Cast, Space","Sunrise, Bandai Visual","Sony Pictures Entertainment, Funimation"
3,4037,https://myanimelist.net/anime/4037/Cowboy_Bebo...,True,"[{'type': 'Default', 'title': 'Cowboy Bebop: Y...",Cowboy Bebop: Yose Atsume Blues,Cowboy Bebop: Session XX - Mish-Mash Blues,カウボーイビバップ よせあつめブルース,"['Yoseatsume Blues', 'Cowboy Bebop Session XX'...",TV Special,Original,...,1998,R - 17+,3.0,27,Sci-Fi,,Sunrise,"Adult Cast, Space",Bandai Visual,
4,16870,https://myanimelist.net/anime/16870/The_Last__...,True,"[{'type': 'Default', 'title': 'The Last: Narut...",The Last: Naruto the Movie,Naruto Shippuden the Movie 7: The Last,THE LAST NARUTO THE MOVIE,"['Naruto Movie 10: Naruto the Movie: The Last,...",Movie,Manga,...,2014,PG-13,2.0,112,"Action, Adventure, Fantasy, Romance",Shounen,Studio Pierrot,,"TV Tokyo, Shueisha",VIZ Media
5,12979,https://myanimelist.net/anime/12979/Naruto_SD_...,True,"[{'type': 'Default', 'title': 'Naruto SD: Rock...",Naruto SD: Rock Lee no Seishun Full-Power Ninden,Naruto Spin-Off: Rock Lee & His Ninja Pals,ナルトSD ロック・リーの青春フルパワー忍伝,[],TV,Manga,...,2013,PG-13,2.0,24,"Action, Comedy",Shounen,Studio Pierrot,Parody,"TV Tokyo, Shueisha",VIZ Media
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
499,52816,https://myanimelist.net/anime/52816/Majo_to_Yajuu,True,"[{'type': 'Default', 'title': 'Majo to Yajuu'}...",Majo to Yajuu,The Witch and the Beast,魔女と野獣,['Witch and the Beast'],TV,Manga,...,2024,R - 17+,3.0,24,"Action, Fantasy",Seinen,Yokohama Animation Laboratory,"Adult Cast, Gore","TBS, Kodansha, DAX Production, NBCUniversal En...",
502,52991,https://myanimelist.net/anime/52991/Sousou_no_...,True,"[{'type': 'Default', 'title': 'Sousou no Frier...",Sousou no Frieren,Frieren: Beyond Journey's End,葬送のフリーレン,"['Frieren at the Funeral', 'Frieren The Slayer']",TV,Manga,...,2024,PG-13,2.0,24,"Adventure, Drama, Fantasy",Shounen,Madhouse,,"Aniplex, Dentsu, Shogakukan-Shueisha Productio...",Crunchyroll
504,57325,https://myanimelist.net/anime/57325/Ramen_Akaneko,True,"[{'type': 'Default', 'title': 'Ramen Akaneko'}...",Ramen Akaneko,Ramen Akaneko,ラーメン赤猫,['Red Cat Ramen'],TV,Manga,...,2024,PG-13,2.0,23,Comedy,Shounen,E&H Production,"Anthropomorphic, Workplace","TBS, Movic, GAGA, Tohokushinsha Film Corporati...",
505,57334,https://myanimelist.net/anime/57334/Dandadan,True,"[{'type': 'Default', 'title': 'Dandadan'}, {'t...",Dandadan,Dan Da Dan,ダンダダン,[],TV,Manga,...,2024,R - 17+,3.0,23,"Action, Comedy, Supernatural",Shounen,Science SARU,,"Aniplex, Dentsu, Shogakukan-Shueisha Productio...",GKIDS


## Drop irrelevant columns

In [238]:
len(df_clean.columns)

73

In [239]:
# drop irrelevant columns 
cols_to_drop = df_clean.columns[df_clean.columns.str.contains('approved|japanese|url|youtube|broadcast|aired', case=False, regex=True)]

df_clean = df_clean.drop(columns=cols_to_drop)

print("Dropped columns:", list(cols_to_drop))
print("Remaining columns:", list(df_clean.columns))

Dropped columns: ['url', 'approved', 'title_japanese', 'images.jpg.image_url', 'images.jpg.small_image_url', 'images.jpg.large_image_url', 'images.webp.image_url', 'images.webp.small_image_url', 'images.webp.large_image_url', 'trailer.youtube_id', 'trailer.url', 'trailer.embed_url', 'trailer.images.image_url', 'trailer.images.small_image_url', 'trailer.images.medium_image_url', 'trailer.images.large_image_url', 'trailer.images.maximum_image_url', 'aired.from', 'aired.to', 'aired.prop.from.day', 'aired.prop.from.month', 'aired.prop.from.year', 'aired.prop.to.day', 'aired.prop.to.month', 'aired.prop.to.year', 'aired.string', 'broadcast.day', 'broadcast.time', 'broadcast.timezone', 'broadcast.string']
Remaining columns: ['mal_id', 'titles', 'title', 'title_english', 'title_synonyms', 'type', 'source', 'episodes', 'status', 'airing', 'duration', 'rating', 'score', 'scored_by', 'rank', 'popularity', 'members', 'favorites', 'synopsis', 'background', 'season', 'year', 'producers', 'licensors'

In [240]:
# Drop duration, year, rating as transformed values have been extracted
df_clean.drop(columns=["duration","rating","year"], inplace=True)

# Drop original JSON-like fields after extracting names
df_clean.drop(columns=["genres", "themes", "studios", "demographics", "producers", "licensors"], inplace=True)

In [241]:
df_clean["explicit_genres"].value_counts()

explicit_genres
[]    424
Name: count, dtype: int64

In [242]:
#drop unused columns 
df_clean = df_clean.drop(columns=['explicit_genres', 'synopsis', 'background', 'season']) #Season could be used for analysis if we want to look at trends over time
print(list(df_clean.columns))

['mal_id', 'titles', 'title', 'title_english', 'title_synonyms', 'type', 'source', 'episodes', 'status', 'airing', 'score', 'scored_by', 'rank', 'popularity', 'members', 'favorites', 'relation_type', 'franchise_id', 'franchise_name', 'start_year', 'end_year', 'rating_cleaned', 'rating_num', 'duration_min', 'genres_clean', 'demographics_clean', 'studios_clean', 'themes_clean', 'producers_clean', 'licensors_clean']


In [243]:
len(df_clean)

424

In [244]:
#fill in demographic vals 
demographic_lookup = {}

for fid, grp in df_clean.groupby('franchise_id'):
    dem_list = []
    
    for entry in grp['demographics_clean']:
        if entry in [np.nan, None, '', []]:
            continue
        if isinstance(entry, list):
            dem_list.extend(d.get('name', '').strip() for d in entry if isinstance(d, dict))
        elif isinstance(entry, str):
            dem_list.extend(e.strip() for e in entry.split(','))

    # Remove empty values
    clean_list = [d for d in dem_list if d]

    if clean_list:
        demographic_lookup[fid] = Counter(clean_list).most_common(1)[0][0]
    else:
        demographic_lookup[fid] = np.nan

# create a new column using this lookup
df_clean["franchise_demographic"] = df_clean["franchise_id"].map(demographic_lookup)

# Fill any remaining NaN values with "NA" string
df_clean["franchise_demographic"] = df_clean["franchise_demographic"].fillna("Unknown")

#drop unnecessary columns
df_clean.drop(columns=["demographics_clean"], inplace=True)

## Kaggle Dataset

In [245]:
#load external dataset
kaggle_df_raw = pd.read_csv("kaggle_anime.csv")

In [246]:
kaggle_df_raw.head()

,MAL_ID,Name,Score,Genres,English name,Japanese name,Type,Episodes,Aired,Premiered,...,Score-10,Score-9,Score-8,Score-7,Score-6,Score-5,Score-4,Score-3,Score-2,Score-1
0,1,Cowboy Bebop,8.78,"Action, Adventure, Comedy, Drama, Sci-Fi, Space",Cowboy Bebop,カウボーイビバップ,TV,26,"Apr 3, 1998 to Apr 24, 1999",Spring 1998,...,229170.0,182126.0,131625.0,62330.0,20688.0,8904.0,3184.0,1357.0,741.0,1580.0
1,5,Cowboy Bebop: Tengoku no Tobira,8.39,"Action, Drama, Mystery, Sci-Fi, Space",Cowboy Bebop:The Movie,カウボーイビバップ 天国の扉,Movie,1,"Sep 1, 2001",Unknown,...,30043.0,49201.0,49505.0,22632.0,5805.0,1877.0,577.0,221.0,109.0,379.0
2,6,Trigun,8.24,"Action, Sci-Fi, Adventure, Comedy, Drama, Shounen",Trigun,トライガン,TV,26,"Apr 1, 1998 to Sep 30, 1998",Spring 1998,...,50229.0,75651.0,86142.0,49432.0,15376.0,5838.0,1965.0,664.0,316.0,533.0
3,7,Witch Hunter Robin,7.27,"Action, Mystery, Police, Supernatural, Drama, ...",Witch Hunter Robin,Witch Hunter ROBIN (ウイッチハンターロビン),TV,26,"Jul 2, 2002 to Dec 24, 2002",Summer 2002,...,2182.0,4806.0,10128.0,11618.0,5709.0,2920.0,1083.0,353.0,164.0,131.0
4,8,Bouken Ou Beet,6.98,"Adventure, Fantasy, Shounen, Supernatural",Beet the Vandel Buster,冒険王ビィト,TV,52,"Sep 30, 2004 to Sep 29, 2005",Fall 2004,...,312.0,529.0,1242.0,1713.0,1068.0,634.0,265.0,83.0,50.0,27.0


In [247]:
kaggle_df_raw[['MAL_ID', 'Score']].isna().sum()

MAL_ID    0
Score     0
dtype: int64

In [248]:
print(kaggle_df_raw['MAL_ID'].value_counts(dropna=False))

MAL_ID
1        1
33089    1
33103    1
33102    1
33099    1
        ..
9673     1
9675     1
9689     1
9690     1
48492    1
Name: count, Length: 17562, dtype: int64


In [249]:
print(kaggle_df_raw['Score'].value_counts(dropna=False))

Score
Unknown    5141
6.48         74
6.3          72
6.31         72
6.52         71
           ... 
3.19          1
4.2           1
4.35          1
3.32          1
4.06          1
Name: count, Length: 533, dtype: int64


In [250]:
len(kaggle_df_raw)

17562

In [251]:
# Remove rows where the Score is "Unknown"
kaggle_df = kaggle_df_raw[kaggle_df_raw["Score"] != "Unknown"].reset_index(drop=True)
kaggle_df['Score'] = pd.to_numeric(kaggle_df['Score'], errors='coerce')
len(kaggle_df)

12421

In [252]:
kaggle_df.drop_duplicates(subset='MAL_ID', inplace=True)
len(kaggle_df)

12421

In [253]:
kaggle_df['Type'].value_counts()

Type
TV         3837
OVA        2999
Movie      2017
Special    1754
ONA        1084
Music       730
Name: count, dtype: int64

In [254]:
kaggle_df = kaggle_df[~kaggle_df['Type'].isin(["Music"])].reset_index(drop=True)
kaggle_df['Type'].value_counts()

Type
TV         3837
OVA        2999
Movie      2017
Special    1754
ONA        1084
Name: count, dtype: int64

In [255]:
len(kaggle_df)

11691

In [256]:
all_genres = kaggle_df["Genres"].dropna().str.split(", ")
unique_genres = sorted(set(genre for sublist in all_genres for genre in sublist))
print(unique_genres)

['Action', 'Adventure', 'Cars', 'Comedy', 'Dementia', 'Demons', 'Drama', 'Ecchi', 'Fantasy', 'Game', 'Harem', 'Hentai', 'Historical', 'Horror', 'Josei', 'Kids', 'Magic', 'Martial Arts', 'Mecha', 'Military', 'Music', 'Mystery', 'Parody', 'Police', 'Psychological', 'Romance', 'Samurai', 'School', 'Sci-Fi', 'Seinen', 'Shoujo', 'Shoujo Ai', 'Shounen', 'Shounen Ai', 'Slice of Life', 'Space', 'Sports', 'Super Power', 'Supernatural', 'Thriller', 'Unknown', 'Vampire', 'Yaoi', 'Yuri']


In [257]:
kaggle_df = kaggle_df[~kaggle_df['Genres'].str.contains("hentai|music|unknown", case=False, na=False)]

In [258]:
len(kaggle_df)

9858

In [259]:
#Remove anything with duration <5 (similar to watchlist dataset)
kaggle_df['duration_min'] = kaggle_df['Duration'].apply(convert_to_minutes)
kaggle_df = kaggle_df[~(kaggle_df["duration_min"] < 5)]

In [260]:
# first, make sure the score‑count columns are numeric
score_cols = [f"Score-{i}" for i in range(1, 11)]
for c in score_cols:
    kaggle_df[c] = pd.to_numeric(kaggle_df[c], errors="coerce").fillna(0)

# total number of ratings for each anime
kaggle_df["n_ratings"] = kaggle_df[score_cols].sum(axis=1)

In [261]:
# drop rows with no score or no ratings
kaggle_df = kaggle_df.dropna(subset=["Score", "n_ratings"])
kaggle_df = kaggle_df[kaggle_df["n_ratings"] > 0]

In [262]:
kaggle_df

,MAL_ID,Name,Score,Genres,English name,Japanese name,Type,Episodes,Aired,Premiered,...,Score-8,Score-7,Score-6,Score-5,Score-4,Score-3,Score-2,Score-1,duration_min,n_ratings
0,1,Cowboy Bebop,8.78,"Action, Adventure, Comedy, Drama, Sci-Fi, Space",Cowboy Bebop,カウボーイビバップ,TV,26,"Apr 3, 1998 to Apr 24, 1999",Spring 1998,...,131625.0,62330.0,20688.0,8904.0,3184.0,1357.0,741.0,1580.0,24,641705.0
1,5,Cowboy Bebop: Tengoku no Tobira,8.39,"Action, Drama, Mystery, Sci-Fi, Space",Cowboy Bebop:The Movie,カウボーイビバップ 天国の扉,Movie,1,"Sep 1, 2001",Unknown,...,49505.0,22632.0,5805.0,1877.0,577.0,221.0,109.0,379.0,115,160349.0
2,6,Trigun,8.24,"Action, Sci-Fi, Adventure, Comedy, Drama, Shounen",Trigun,トライガン,TV,26,"Apr 1, 1998 to Sep 30, 1998",Spring 1998,...,86142.0,49432.0,15376.0,5838.0,1965.0,664.0,316.0,533.0,24,286146.0
3,7,Witch Hunter Robin,7.27,"Action, Mystery, Police, Supernatural, Drama, ...",Witch Hunter Robin,Witch Hunter ROBIN (ウイッチハンターロビン),TV,26,"Jul 2, 2002 to Dec 24, 2002",Summer 2002,...,10128.0,11618.0,5709.0,2920.0,1083.0,353.0,164.0,131.0,25,39094.0
4,8,Bouken Ou Beet,6.98,"Adventure, Fantasy, Shounen, Supernatural",Beet the Vandel Buster,冒険王ビィト,TV,52,"Sep 30, 2004 to Sep 29, 2005",Fall 2004,...,1242.0,1713.0,1068.0,634.0,265.0,83.0,50.0,27.0,23,5923.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11685,46118,Wave!!: Surfing Yappe!! (TV),6.05,"Slice of Life, Sports",WAVE!! -Let's go surfing!!-,WAVE!!〜サーフィンやっぺ!!〜,TV,12,"Jan 12, 2021 to ?",Winter 2021,...,392.0,723.0,782.0,570.0,335.0,172.0,77.0,49.0,23,3548.0
11686,46491,Santa Company: Manatsu no Merry Christmas,5.68,Fantasy,SANTA COMPANY ~Midsummer Merry Christmas~,サンタ・カンパニー ～真夏のメリークリスマス～,Movie,1,"Jan 28, 2021",Unknown,...,51.0,139.0,212.0,195.0,101.0,48.0,29.0,10.0,30,827.0
11688,47614,Nu Wushen de Canzhuo Spring Festival Special,6.83,"Slice of Life, Comedy",Cooking with Valkyries Spring Festival Special,Unknown,Special,1,"Feb 11, 2021",Unknown,...,16.0,29.0,26.0,10.0,1.0,0.0,3.0,0.0,10,117.0
11689,47616,Yakusoku no Neverland 2nd Season: Michishirube,4.81,"Mystery, Psychological, Supernatural, Thriller...",The Promised Neverland Season 2 Episode 5.5,約束のネバーランド 特別編「道標」,Special,1,"Feb 12, 2021",Unknown,...,317.0,565.0,998.0,1542.0,831.0,516.0,336.0,722.0,23,6156.0


In [263]:
# define prior mean C and pseudo‑votes m 
C = (kaggle_df['Score'] * kaggle_df['n_ratings']).sum() / kaggle_df['n_ratings'].sum()
m = kaggle_df['n_ratings'].median()

# calculate the Bayesian‑shrunk score per row 
df_clean['shrunk_score'] = round((
    df_clean['score']     * df_clean['scored_by']  # observed total points
  + m * C                                          # m pseudo‑points at the global mean
) / (df_clean['scored_by'] + m), 2)                # total votes including pseudo‑votes

# compare raw vs shrunk 
comparison = df_clean[['title','score','scored_by','shrunk_score']].copy()
comparison['delta'] = comparison['shrunk_score'] - comparison['score']

print("First 10 comparisons:")
print(comparison.head(10))

print("\nTop 10 largest adjustments:")
print(
    comparison
    .reindex(comparison['delta'].abs().sort_values(ascending=False).index)
    .head(10))

First 10 comparisons:
                                                title  score  scored_by  \
0                                        Cowboy Bebop   8.75  1029578.0   
1                     Cowboy Bebop: Tengoku no Tobira   8.38   226903.0   
3                     Cowboy Bebop: Yose Atsume Blues   7.41    20054.0   
4                          The Last: Naruto the Movie   7.79   430039.0   
5    Naruto SD: Rock Lee no Seishun Full-Power Ninden   7.17    52180.0   
6   Naruto: Shippuuden - Shippuu! "Konoha Gakuen" Den   7.13    33103.0   
7                                         Naruto x UT   7.36    26526.0   
8              Naruto: Shippuuden - Sunny Side Battle   7.58    42106.0   
10         Naruto: Shippuuden Movie 6 - Road to Ninja   7.69   221032.0   
11                           Boruto: Naruto the Movie   7.38   312829.0   

    shrunk_score  delta  
0           8.75   0.00  
1           8.37  -0.01  
3           7.45   0.04  
4           7.79   0.00  
5           7.20   0.0

In [264]:
df_clean.isna().sum()

mal_id                    0
titles                    0
title                     0
title_english            63
title_synonyms            0
type                      0
source                    0
episodes                  0
status                    0
airing                    0
score                     0
scored_by                 0
rank                      0
popularity                0
members                   0
favorites                 0
relation_type             0
franchise_id              0
franchise_name            0
start_year                0
end_year                  0
rating_cleaned            0
rating_num                0
duration_min              0
genres_clean              0
studios_clean             0
themes_clean              0
producers_clean           0
licensors_clean           0
franchise_demographic     0
shrunk_score              0
dtype: int64

In [265]:
df_clean

,mal_id,titles,title,title_english,title_synonyms,type,source,episodes,status,airing,...,rating_cleaned,rating_num,duration_min,genres_clean,studios_clean,themes_clean,producers_clean,licensors_clean,franchise_demographic,shrunk_score
0,1,"[{'type': 'Default', 'title': 'Cowboy Bebop'},...",Cowboy Bebop,Cowboy Bebop,[],TV,Original,26.0,Finished Airing,False,...,R - 17+,3.0,24,"Action, Award Winning, Sci-Fi",Sunrise,"Adult Cast, Space","Bandai Visual, Victor Entertainment, Audio Pla...",Funimation,Unknown,8.75
1,5,"[{'type': 'Default', 'title': 'Cowboy Bebop: T...",Cowboy Bebop: Tengoku no Tobira,Cowboy Bebop: The Movie,"[""Cowboy Bebop: Knockin' on Heaven's Door""]",Movie,Original,1.0,Finished Airing,False,...,R - 17+,3.0,115,"Action, Sci-Fi",Bones,"Adult Cast, Space","Sunrise, Bandai Visual","Sony Pictures Entertainment, Funimation",Unknown,8.37
3,4037,"[{'type': 'Default', 'title': 'Cowboy Bebop: Y...",Cowboy Bebop: Yose Atsume Blues,Cowboy Bebop: Session XX - Mish-Mash Blues,"['Yoseatsume Blues', 'Cowboy Bebop Session XX'...",TV Special,Original,1.0,Finished Airing,False,...,R - 17+,3.0,27,Sci-Fi,Sunrise,"Adult Cast, Space",Bandai Visual,,Unknown,7.45
4,16870,"[{'type': 'Default', 'title': 'The Last: Narut...",The Last: Naruto the Movie,Naruto Shippuden the Movie 7: The Last,"['Naruto Movie 10: Naruto the Movie: The Last,...",Movie,Manga,1.0,Finished Airing,False,...,PG-13,2.0,112,"Action, Adventure, Fantasy, Romance",Studio Pierrot,,"TV Tokyo, Shueisha",VIZ Media,Shounen,7.79
5,12979,"[{'type': 'Default', 'title': 'Naruto SD: Rock...",Naruto SD: Rock Lee no Seishun Full-Power Ninden,Naruto Spin-Off: Rock Lee & His Ninja Pals,[],TV,Manga,51.0,Finished Airing,False,...,PG-13,2.0,24,"Action, Comedy",Studio Pierrot,Parody,"TV Tokyo, Shueisha",VIZ Media,Shounen,7.20
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
499,52816,"[{'type': 'Default', 'title': 'Majo to Yajuu'}...",Majo to Yajuu,The Witch and the Beast,['Witch and the Beast'],TV,Manga,12.0,Finished Airing,False,...,R - 17+,3.0,24,"Action, Fantasy",Yokohama Animation Laboratory,"Adult Cast, Gore","TBS, Kodansha, DAX Production, NBCUniversal En...",,Seinen,7.45
502,52991,"[{'type': 'Default', 'title': 'Sousou no Frier...",Sousou no Frieren,Frieren: Beyond Journey's End,"['Frieren at the Funeral', 'Frieren The Slayer']",TV,Manga,28.0,Finished Airing,False,...,PG-13,2.0,24,"Adventure, Drama, Fantasy",Madhouse,,"Aniplex, Dentsu, Shogakukan-Shueisha Productio...",Crunchyroll,Shounen,9.28
504,57325,"[{'type': 'Default', 'title': 'Ramen Akaneko'}...",Ramen Akaneko,Ramen Akaneko,['Red Cat Ramen'],TV,Manga,12.0,Finished Airing,False,...,PG-13,2.0,23,Comedy,E&H Production,"Anthropomorphic, Workplace","TBS, Movic, GAGA, Tohokushinsha Film Corporati...",,Shounen,7.50
505,57334,"[{'type': 'Default', 'title': 'Dandadan'}, {'t...",Dandadan,Dan Da Dan,[],TV,Manga,12.0,Finished Airing,False,...,R - 17+,3.0,23,"Action, Comedy, Supernatural",Science SARU,,"Aniplex, Dentsu, Shogakukan-Shueisha Productio...",GKIDS,Shounen,8.46


## Aggregated Franchise Data

In [266]:
df_franchise = df_clean.copy()

In [267]:
# helper functions
def list_unique(series, splitter=None):
    items = set()
    for entry in series.dropna():
        if splitter:
            items.update(i.strip() for i in str(entry).split(splitter))
        else:
            items.add(entry)
    return sorted(items)

def genre_by_frequency(series, splitter=','):
    counter = Counter()
    for entry in series.dropna():
        counter.update(i.strip() for i in str(entry).split(splitter))
    return [g for g,_ in sorted(counter.items(), key=lambda x:(-x[1],x[0]))]

def primary_genres(series, top_n=2, splitter=','):
    counter = Counter()
    for entry in series.dropna():
        counter.update(i.strip() for i in str(entry).split(splitter))
    return [g for g,_ in counter.most_common(top_n)]

def pick_by_year(series, years, mode='latest'):
    idx = years.idxmax() if mode=='latest' else years.idxmin()
    return series.loc[idx]

def most_common_or_earliest(x, years):
    # 1. Try most common non-null value
    counts = x.value_counts(dropna=True)
    if not counts.empty and counts.iloc[0] > 1:
        return counts.index[0]
    # 2. Otherwise, return the earliest non-null value
    non_null = x[~x.isna()]
    if not non_null.empty:
        idx = years[~x.isna()].idxmin()
        return x.loc[idx]
    # 3. Else, return null (np.nan)
    return np.nan

def status_and_num(statuses, status_nums, types):
    s = statuses.str.strip().str.lower()
    t = types.str.strip().str.upper()
    mask_current = (s == "currently airing") & (t == "TV")
    if mask_current.any():
        idx = mask_current[mask_current].index[0]
        return "Currently Airing", status_nums.loc[idx]
    mask_finished = s == "finished airing"
    if mask_finished.any():
        idx = mask_finished[mask_finished].index[0]
        return "Finished Airing", status_nums.loc[idx]
    return np.nan, np.nan

In [268]:
# summarize every franchise 
def summarize_franchise(g):
    fid   = g['franchise_id'].iloc[0]
    name  = g['franchise_name'].iloc[0]
    yrs   = g['start_year']
    types = g["type"]
    
    status_val, status_num_val = status_and_num(g["status"], g["airing"], types)

    return pd.Series({
        'franchise_id':     g['franchise_id'].iloc[0],
        'franchise_name':    g['franchise_name'].iloc[0],
        'MAL_IDs':          list_unique(g['mal_id']),
        'Start_Year':       int(yrs.min()),
        'Status':           status_val,
        'Airing':           status_num_val,
        'Genres':           genre_by_frequency(g['genres_clean']),
        'Primary_Genres':   primary_genres(g['genres_clean'], top_n=2),
        'Themes':           genre_by_frequency(g['themes_clean']),
        'Demographic':      most_common_or_earliest(g['franchise_demographic'], yrs),
        'Studio':           most_common_or_earliest(g['studios_clean'], yrs),
        'Avg_Weighted_Score': round(g['shrunk_score'].mean(), 2),
        'Best_Rank':        int(g['rank'].min()),
        'Top_Popularity':   int(g['popularity'].min()),
        'Types':            list_unique(g['type']),
        'Source':           pick_by_year(g['source'], yrs, 'earliest'),
        "Rating":           pick_by_year(g["rating_cleaned"], yrs, 'earliest'),
        "Rating_Num":       pick_by_year(g["rating_num"], yrs, 'earliest'),
    })

franchise_agg = (
    df_franchise
      .groupby('franchise_id', group_keys=False)
      .apply(summarize_franchise)
      .reset_index(drop=True))

/var/folders/r3/20vxfkdn06q1gs5plztt2_bm0000gn/T/ipykernel_8518/2327903162.py:34: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(summarize_franchise)


In [269]:
def clean_list_field(df, field):
    def clean(x):
        if x is None:
            return 'Unknown'
        if not isinstance(x, list):
            if pd.isna(x):
                return 'Unknown'
            if isinstance(x, str):
                if x.strip() in ["[]", "['']", '']:
                    return 'Unknown'
                return x  
        
        if isinstance(x, list):
            if not x or all((str(i).strip() == '' for i in x)):
                return 'Unknown'
            return ', '.join(str(i).strip() for i in x if str(i).strip())
        return str(x)
    
    return df[field].apply(clean)

In [270]:
#Apply to cols
for col in ['MAL_IDs', 'Genres', 'Themes', 'Types', 'Primary_Genres']:
    franchise_agg[col] = clean_list_field(franchise_agg, col)

In [271]:
#Reorder cols
cols = [
  'franchise_id','franchise_name','Primary_Genres', 'Genres',
  'Demographic','Themes',
  'Avg_Weighted_Score','Best_Rank','Top_Popularity',
  'Studio','Rating','Rating_Num',
  'Status','Airing','Start_Year','Source','Types',
  'MAL_IDs',
]
franchise_agg = franchise_agg[cols]
franchise_agg.head()

,franchise_id,franchise_name,Primary_Genres,Genres,Demographic,Themes,Avg_Weighted_Score,Best_Rank,Top_Popularity,Studio,Rating,Rating_Num,Status,Airing,Start_Year,Source,Types,MAL_IDs
0,1,Cowboy Bebop,"Sci-Fi, Action","Sci-Fi, Action, Award Winning",Unknown,"Adult Cast, Space",8.19,47,42,Sunrise,R - 17+,3.0,Finished Airing,False,1998,Original,"Movie, TV, TV Special","1, 5, 4037"
1,20,Naruto,"Action, Fantasy","Action, Fantasy, Adventure, Comedy, Gourmet, R...",Shounen,"Martial Arts, Isekai, Parody, School, Time Travel",7.23,318,9,Studio Pierrot,PG-13,2.0,Finished Airing,False,2002,Manga,"Movie, OVA, Special, TV","20, 442, 594, 761, 936, 1074, 1735, 2144, 2248..."
2,21,One Piece,"Fantasy, Adventure","Fantasy, Adventure, Action, Comedy, Drama, Spo...",Shounen,"Team Sports, Historical, Idols (Female)",7.46,8,17,Toei Animation,PG-13,2.0,Currently Airing,True,1998,Manga,"Movie, ONA, OVA, Special, TV, TV Special","21, 459, 460, 461, 462, 463, 464, 465, 466, 10..."
3,120,Fruits Basket,"Drama, Romance","Drama, Romance, Supernatural",Shoujo,"School, Love Polygon",8.33,15,210,TMS Entertainment,PG-13,2.0,Finished Airing,False,2001,Manga,"Movie, TV","120, 38680, 40417, 42938, 49310"
4,121,Fullmetal Alchemist,"Fantasy, Adventure","Fantasy, Adventure, Drama, Action, Award Winni...",Shounen,"Military, Historical",7.80,2,3,Bones,R - 17+,3.0,Finished Airing,False,2003,Manga,"Movie, OVA, Special, TV","121, 430, 664, 908, 5114, 6421, 9135"


In [ ]:
franchise_agg

In [274]:
franchise_agg.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59 entries, 0 to 58
Data columns (total 18 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   franchise_id        59 non-null     int64  
 1   franchise_name      59 non-null     object 
 2   Primary_Genres      59 non-null     object 
 3   Genres              59 non-null     object 
 4   Demographic         59 non-null     object 
 5   Themes              59 non-null     object 
 6   Avg_Weighted_Score  59 non-null     float64
 7   Best_Rank           59 non-null     int64  
 8   Top_Popularity      59 non-null     int64  
 9   Studio              59 non-null     object 
 10  Rating              59 non-null     object 
 11  Rating_Num          59 non-null     float64
 12  Status              59 non-null     object 
 13  Airing              59 non-null     bool   
 14  Start_Year          59 non-null     int64  
 15  Source              59 non-null     object 
 16  Types     

In [273]:
#export dataset
franchise_agg.to_csv("franchise_agg.csv", index=False)